데이터 전처리 및 클리어링이 모두 끝나있는 csv파일이 있다는 다 있다는 가정하에 모델 코드만 작성 (원핫 인코딩?사용안할거같기는함)
*classifiermixin

In [1]:
!pip install klib
!pip install vecstack
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 9.8 MB/s 
  Attempting uninstall: Jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires Jinja2<3.0,>=2.10.1, but you have jinja2 3.1.2 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19877 sha256=ae9e54ecb05d773dede75aa4090645bff3635ce4d4136a43f000e3ef0d0fe022
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e

In [2]:
# Data Wrangling
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import seaborn as sns
%matplotlib inline

# EDA
import klib

# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from bayes_opt import BayesianOptimization

# Modeling
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingClassifier
from xgboost import XGBRegressor
from xgboost import plot_importance
import lightgbm as lgb
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error
from itertools import combinations
import catboost
from catboost import Pool
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import ensemble, metrics

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Utility
import os
import time
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
from vecstack import stacking

SyntaxError: ignored

# 1. Read data

In [ ]:
# 학습용과 평가용(제출용) 데이터를 읽어들인다.
#train = pd.read_csv('', encoding='cp949')
#test = pd.read_csv('', encoding='cp949')

# 학습용 정답 데이터를 읽는다.
#y_train = pd.read_csv('', encoding='cp949').gender #<- 성별 별로 결과를 알고싶었던 코드파일이라서 정답 데이터뒤에 gender라는 것 삽입

# 전처리를 동일하게 적용하기 위해 두 데이터를 합한다.
#df = pd.concat([train, test]).reset_index(drop=True)
#df                                                     <- 이 부분은 데이터가 완성되면 수정될 부분

1-2 속성간 유사성 시각화(corr)

In [ ]:
klib.corr_plot(df, annot=False, figsize=(15,12))
klib.corr_plot(df, split='high', annot=False, figsize=(15,12))

In [ ]:
klib.corr_plot(pd.concat([train, y_train], axis=1), target='알고싶은 열') #알고싶은 열과 나머지 속성들이 어느정도의 상관성을 가지는지 시각화

In [ ]:
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.3, random_state=0) #데이터 분할 학습용 70% 검증용 30%

In [ ]:
logreg = LogisticRegression(random_state=0, n_jobs=-1)
rf = RandomForestRegressor(random_state=0, n_jobs=-1)
kr = KNeighborsRegressor(random_state=0, n_jobs=-1)
gbm = GradientBoostingRegressor(random_state=0)
lgbm = LGBMClassifier(random_state=0 ,n_jobs=-1)
xgb = XGBRegressor(random_state=0 ,n_jobs=-1)
lr = LinearRegression(random_state=0 ,n_jobs=-1)
mlp = MLPClassifier(random_state=0 ,n_jobs=-1)
cb = CatBoostRegressor(random_state=0 ,n_jobs=-1)
svr_poly = SVR(kernel='poly', random_state=0 ,n_jobs=-1)
svr_lin = SVR(kernel='linear', random_state=0 ,n_jobs=-1)
svr_rbf = SVR(kernel='rbf', random_state=0 ,n_jobs=-1)

models = [logreg, rf, kr, gbm, lgbm, xgb.lr.mlp,svm]

In [ ]:
for model in models:
    
    lucky_seed = [2533,5066,9538]
    
    cv_scores = []
    
    for rs in lucky_seed:
        
        skf = StratifiedKFold(n_splits=4 , shuffle=False, random_state=rs)
        
        scores = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv = skf)
        
        cv_scores.append(scores)
        
    print(f'{model.__class__.__name__}, \ncv성능들: {cv_scores}\n최고성능: {max([score for scoreArr in cv_scores for score in scoreArr])}\n평균성능: {np.mean(cv_scores)}\n')

모델별 하이퍼 파라미터 조정 성능을 높이기 위한 세세한 파라미터 조정 

+ 하이퍼 파라미터를 자동적으로 찾아주는 기법

In [ ]:
# MLPClassifier
param_grid={'batch_size': ['auto'],
         'learning_rate' : ['constant'],
         'activation': ['tanh'],
         'solver': ['sgd'],
           'alpha' : [0.001]}

start = time.time()
rand_search = RandomizedSearchCV(MLPClassifier(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=3, random_state=0, n_jobs=-1)
rand_search.fit(X_train, y_train)
score = rand_search.score(X_dev, y_dev)
print('{:30f} {:.1f}'.format(score, time.time() - start))

print(rand_search.best_params_)
print('-'*50)

# LogisticRegression
param_grid={'C': [0.007],      
         'penalty': ['l2']}

start = time.time()
rand_search = RandomizedSearchCV(LogisticRegression(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=3, random_state=0, n_jobs=-1)
rand_search.fit(X_train, y_train)
score = rand_search.score(X_dev, y_dev)
print('{:30f} {:.1f}'.format(score, time.time() - start))

print(rand_search.best_params_)
print('-'*50)

# CatBoostRegressor
param_grid={'iterations':[4],
            'learning_rate':[0.1],
            'depth':[4]}

start = time.time()
rand_search = RandomizedSearchCV(CatBoostRegressor(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=3, random_state=0, n_jobs=-1)
rand_search.fit(X_train, y_train)
score = rand_search.score(X_dev, y_dev)
print('{:30f} {:.1f}'.format(score, time.time() - start))

print(rand_search.best_params_)
print('-'*50)

# LinearRegression
#param_grid={}}

#start = time.time()
#rand_search = RandomizedSearchCV(LinearRegression(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=3, random_state=0, n_jobs=-1)
#rand_search.fit(X_train, y_train)
#score = rand_search.score(X_dev, y_dev)
#print('{:30f} {:.1f}'.format(score, time.time() - start))

#print(rand_search.best_params_)
#print('-'*50)

# XGBRegressor
param_grid={'nthread':[4],
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07],
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

start = time.time()
rand_search = RandomizedSearchCV(XGBRegressor(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=3, random_state=0, n_jobs=-1)
rand_search.fit(X_train, y_train)
score = rand_search.score(X_dev, y_dev)
print('{:30f} {:.1f}'.format(score, time.time() - start))

print(rand_search.best_params_)
print('-'*50)

# GradientBoostingRegressor
param_grid={'loss': ['absolute_error'],#squared_error      
         'learning_rate': ['0.001'],
         'n_estimators':[300],
         'criterion':['mse'],
         'max_depth':[9],
         'max_features':['auto'],
         'validation_fraction ':[0.2]}

start = time.time()
rand_search = RandomizedSearchCV(GradientBoostingRegressor(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=3, random_state=0, n_jobs=-1)
rand_search.fit(X_train, y_train)
score = rand_search.score(X_dev, y_dev)
print('{:30f} {:.1f}'.format(score, time.time() - start))

print(rand_search.best_params_)
print('-'*50)

# KNeighborsRegressor
param_grid={'n_neighbors ': [7],      
            'weights ': ['distance'],
            'algorithm':['auto']}

start = time.time()
rand_search = RandomizedSearchCV(KNeighborsRegressor(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=3, random_state=0, n_jobs=-1)
rand_search.fit(X_train, y_train)
score = rand_search.score(X_dev, y_dev)
print('{:30f} {:.1f}'.format(score, time.time() - start))

print(rand_search.best_params_)
print('-'*50)

# RandomForestRegressor
param_grid={'n_estimators': [300],
            'max_depth': [30],
            'min_samples_split':[5],
            'min_samples_leaf':[2],
            'min_weight_fraction_leaf':['auto'],
            'max_features':['auto'],
            'max_leaf_nodes':['None']}

start = time.time()
rand_search = RandomizedSearchCV(RandomForestRegressor(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=3, random_state=0, n_jobs=-1)
rand_search.fit(X_train, y_train)
score = rand_search.score(X_dev, y_dev)
print('{:30f} {:.1f}'.format(score, time.time() - start))

print(rand_search.best_params_)
print('-' * 50)

# LGBMClassifier
param_grid={'n_estimators': [16],
            'learning_rate': [0.05],
            'max_depth': [5],
            'metric ':['mse'],
            'application':['regression'],
            'min_data_in_leaf ':[20],
            'num_leaves ':[30]}#mae
start = time.time()
rand_search = RandomizedSearchCV(LGBMClassifier(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=skfold, random_state=0, n_jobs=-1)
rand_search.fit(X_train, y_train)
score = rand_search.score(X_dev, y_dev)
print('{:30f} {:.1f}'.format(score, time.time() - start))

print(rand_search.best_params_)
print('-'*50)

# svr_poly 
param_grid= {'kernel': ['poly'],
            'C': [100],
            'gamma': ['auto'],
             'degree':[3],
             'epsilon':[0.1],
             'coef0':[1]}
start = time.time()
rand_search = RandomizedSearchCV(SVR(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=skfold, random_state=0, n_jobs=-1)
rand_search.fit(X_train, y_train)
score = rand_search.score(X_dev, y_dev)
print('{:30f} {:.1f}'.format(score, time.time() - start))

print(rand_search.best_params_)
print('-'*50)

# svr_lin 
param_grid={'kernel': ['linear'],
            'C': [100],
            'gamma': ['auto']}
start = time.time()
rand_search = RandomizedSearchCV(SVR(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=3, random_state=0, n_jobs=-1)
rand_search.fit(X_train, y_train)
score = rand_search.score(X_dev, y_dev)
print('{:30f} {:.1f}'.format(score, time.time() - start))

print(rand_search.best_params_)
print('-'*50)

# svr_rbf 
param_grid={'kernel': ['rbf'],
            'C': [100],
            'gamma': [0.1],
            'epsilon': [0.1]}

start = time.time()
rand_search = RandomizedSearchCV(SVR(random_state=0), param_grid, n_iter=20 ,scoring='roc_auc', 
                                     cv=skfold, random_state=0, n_jobs=-1)
rand_search.fit(X_train, y_train)
score = rand_search.score(X_dev, y_dev)
print('{:30f} {:.1f}'.format(score, time.time() - start))

print(rand_search.best_params_)
print('-'*50)



In [ ]:
clfs = [
    (
        MLPClassifier(random_state=0),
        {'batch_size': ['auto'],
         'learning_rate' : ['constant'],
         'activation': ['tanh'],
         'solver': ['sgd'],
         'alpha' : [0.001]}
    ),    
    (
        LogisticRegression(random_state=0),  
        {'C': [0.007],      
         'penalty': ['l2']}
    ),
     (
        XGBRegressor(random_state=0),
       {'nthread':[4],
        'objective':['reg:linear'],
        'learning_rate': [0.05],
        'max_depth': [6],
        'min_child_weight': [4],
        'silent': [1],
        'subsample': [0.7],
        'colsample_bytree': [0.7],
        'n_estimators': [500]}
    ),    
    (
        LGBMClassifier(random_state=0),
        {'n_estimators': [16],
         'learning_rate': [0.05],
         'max_depth': [5],
         'metric ':['mse'],
         'application':['regression'],
         'min_data_in_leaf ':[20],
         'num_leaves ':[30]}  
    ),
    (
        GradientBoostingRegressor(random_state=0),
        {'loss': ['absolute_error'],#squared_error      
         'learning_rate': ['0.001'],
         'n_estimators':[300],
         'criterion':['mse'],
         'max_depth':[9],
         'max_features':['auto'],
         'validation_fraction ':[0.2]}
    ),
    (
        SVR(random_state=0),
        {'kernal' : ['rbf'],
          'C': [100],
          'gamma': [0.1],
          'epsilon': [0.1]}
    ),
    (
        SVR(random_state=0),
        {'kernal' : ['linear'],
         'C': [100],
         'gamma': ['auto']}
    ),
    (
        SVR(random_state=0),
        {'kernal' : ['poly'],
         'C': [100],
         'gamma': ['auto'],
         'degree':[3],
         'epsilon':[0.1],
         'coef0':[1]}
    ),
    (
        CatBoostRegressor(random_state=0),
        {('iterations':4,
          'learning_rate':0.1,
          'depth':4)}
    ),
    (
        KNeighborsRegressor(random_state=0)
        {'n_neighbors ': [7],      
         'weights ': ['distance'],
         'algorithm':['auto']
        }
    ),
    (
        RandomForestRegressor(random_state=0)
        {'n_estimators': [300],
         'max_depth': [30],
         'min_samples_split':[5],
         'min_samples_leaf':[2],
         'min_weight_fraction_leaf':['auto'],
         'max_features':['auto'],
         'max_leaf_nodes':['None']  
        }
    )
    #(
       #LinearRegression(random_state=0)
    #{}),    
]



clfs_tuned = []  # 튜닝된 모델을 저장
for clf, param_grid in tqdm(clfs):
    start = time.time()
    rand_search = RandomizedSearchCV(clf, param_grid, n_iter=20, scoring='roc_auc', 
                                     cv=3, random_state=0, n_jobs=-1)
    rand_search.fit(X_train, y_train)
    clf_name = type(clf).__name__
    clf_score = rand_search.score(X_dev, y_dev)
    print('{:30s} {:30f} {:.1f}'.format(clf_name, clf_score, time.time() - start))
    clfs_tuned.append((clf_name, rand_search, clf_score))

In [ ]:
ts = time.time()

model = XGBRegressor(
    max_depth=8,
    n_estimators=300,
    min_child_weight=300, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train, 
    y_train, 
    eval_metric='rmse', 
    eval_set=[(X_train, y_train), (X_valid, y_valid)], 
    verbose=True,
    early_stopping_rounds = 10)

time.time() - ts

In [ ]:
results = model.evals_result()
epochs = len(results['validation_1']['rmse'])
x_axis = range(0, epochs)
# plot rmse
fig, ax = plt.subplots()
ax.plot(x_axis, results['validation_0']['rmse'], label='Train')
ax.plot(x_axis, results['validation_1']['rmse'], label='Valid')
ax.legend()
plt.ylabel('RMSE')
plt.title('XGBoost RMSE')
plt.show()

앙상블을 위한 모델들간의 상관관계 출력

In [ ]:
pred_results = []
for name, clf, clf_score in clfs_tuned:
    pred = clf.predict_proba(X_dev)[:,1]
    name = f'{name} \n({clf_score:.4f})'
    pred_results.append(pd.Series(pred, name=name))
ensemble_results = pd.concat(pred_results, axis=1)

# 모형의 예측값 간의 상관관계를 보기 위해 hitmap을 도식한다.
plt.figure(figsize = (8,6))
g = sns.heatmap(ensemble_results.corr(), annot=True, cmap='Blues')
g.set_title("Correlation between models")
plt.show()

In [ ]:
corr = (ensemble_results.corr().sum()-1)/(ensemble_results.corr().shape[0]-1)
names = corr.index
aucs = np.array(corr.index.str[-7:-1]).astype(float)
df = pd.DataFrame({'model': names, 'auc': aucs, 'cor': corr})        

plt.figure(figsize=(8,6))
g = sns.scatterplot(x="cor", y="auc", data=df, s=40, color='red')
for line in range(0, df.shape[0]):
     g.text(df.cor[line]+0.003, df.auc[line]-0.003, 
            df.model[line], horizontalalignment='left', 
            size='medium', color='black', weight='semibold')
        
plt.xlim((df.cor.min()-0.01,df.cor.max()+0.01))
plt.ylim((df.auc.min()-0.01,df.auc.max()+0.01))
plt.xlabel('Mean Agreement')
plt.ylabel('ROC-AUC')
plt.grid()
plt.show()

averaging 앙상블

In [ ]:
# Averaging 앙상블에 사용하지 않을 모델은 주석 처리하시오.
selected = [
    'MLPClassifier',
    'LogisticRegression', 
    'RandomForestRegressor', 
    'KNeighborsRegressor',
    'GradientBoostingRegression',
    'LGBMClassifier',
    'XGBRegressor',
    'LinearRegression',
    'CatBoostRegressor',
    'svr_poly',
    'svr_linear',
    'svr_rbf'

]
models_for_ensemble = [clf for clf in clfs_tuned if clf[0] in selected]

In [ ]:
selected = [ 
    'XGBClassifier', 
    'GradientBoostingClassifier',
    'LogisticRegression'
]
models_for_ensemble = [clf for clf in clfs_tuned if clf[0] in selected]

In [ ]:
# 아래 코드는 추후 estimator class("BlendingClassifier")로 만들 예정

max_score = 0
for p in tqdm([0, 1, 2.56]):  # p==1:산술평균, p=0:기하평균, 그 외:멱평균(주의:멱평균은 과적합 가능성이 높음)    
    for i in range(2, len(models_for_ensemble)+1):
        for models in combinations(models_for_ensemble, i):
            if p == 0:
                pred_mean = gmean([clf.predict_proba(X_dev)[:,1] for name, clf, _ in models], axis=0)
            else:
                preds = [clf.predict_proba(X_dev)[:,1] for name, clf, _ in models]
                pred_mean = (np.sum(np.array(preds)**p, axis=0) / len(models))**(1/p)
            score = roc_auc_score(y_dev, pred_mean)
            if max_score < score:
                best_avg_ensemble = (p, models, score)
                max_score = score

p, models, score = best_avg_ensemble
print('p={}\n{}\n{}'.format(p, '●'.join([clf_name for clf_name, _, _ in models]), score))

스태킹 기법

In [ ]:
# 2-layer stacking

# Initialize 1st level estimators
# 사용하지 않을 모델은 주석 처리하세요.
selected = [
    'MLPClassifier',
    'LogisticRegression', 
    'RandomForestRegressor', 
    'KNeighborsRegressor',
    'GradientBoostingRegression',
    'LGBMClassifier',
    'XGBRegressor',
    'LinearRegression',
    'CatBoostRegressor',
    'svr_poly',
    'svr_linear',
    'svr_rbf'    
]

estimators = [(name, clf) for name, clf, _ in clfs_tuned if name in selected]
stk_clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression(), cv=3)

stk_clf.fit(X_train, y_train)
print(roc_auc_score(y_dev, stk_clf.predict_proba(X_dev)[:,1])) 

In [ ]:
# Apply the best averaging ensemble model to Unseen Data
if p == 0:
    pred_mean = gmean([clf.predict_proba(X_test)[:,1] for name, clf, _ in models], axis=0)
else:
    preds = [clf.predict_proba(X_test)[:,1] for name, clf, _ in models]
    pred_mean = (np.sum(np.array(preds)**p, axis=0) / len(models))**(1/p)
pd.DataFrame({'cust_id': test_id, 'gender': pred_mean}).to_csv('submission_avg.csv', index=False)

# Apply the stacking model to Unseen Data
pd.DataFrame({'cust_id': test_id, 'gender': stk_clf.predict_proba(X_test)[:,1]}).to_csv('submission_stk.csv', index=False)